# Piping a Prompt, Model, and an Output Parser

In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [5]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [6]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [10]:
chat_template = ChatPromptTemplate.from_messages([('human', "I've recently adopted a {pet},can you suggest some {pet} names? \n" +list_instructions)])

In [11]:
print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet},can you suggest some {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [12]:
chat = ChatOpenAI(model='gpt-4o-mini', seed=365, temperature=0, max_tokens=100)


In [13]:
list_output_parser = CommaSeparatedListOutputParser()

In [18]:
chat_template_result = chat_template.invoke({'pet':'dog'})

In [19]:
chat_result = chat.invoke(chat_template_result)

In [20]:
list_output_parser.invoke(chat_result)

['Max',
 'Bella',
 'Charlie',
 'Luna',
 'Rocky',
 'Daisy',
 'Cooper',
 'Sadie',
 'Buddy',
 'Molly',
 'Duke',
 'Ruby',
 'Oliver',
 'Chloe',
 'Jack',
 'Zoey',
 'Bear',
 'Sophie',
 'Tucker',
 'Lily']

In [21]:
chain = chat_template | chat | list_output_parser

In [22]:
chain

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet},can you suggest some {pet} names? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021582A07F40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021582F4F7C0>, root_client=<openai.OpenAI object at 0x00000215829DB8E0>, root_async_client=<openai.AsyncOpenAI object at 0x0000021582A04FD0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), seed=365, max_tokens=100)
| CommaSeparatedListOutputParser()

In [24]:
chain.invoke({'pet':'dog'})

['Max',
 'Bella',
 'Charlie',
 'Luna',
 'Cooper',
 'Daisy',
 'Rocky',
 'Sadie',
 'Buddy',
 'Molly',
 'Duke',
 'Ruby',
 'Oliver',
 'Chloe',
 'Jack',
 'Zoey',
 'Bear',
 'Sophie',
 'Tucker',
 'Lily']

## Batching

In [25]:
chat_template = ChatPromptTemplate.from_messages([('human', "I've recently adopted a {pet}, which is a {breed}. Could you suggest some training tips?? \n")])

In [26]:
chain = chat_template | chat

In [30]:
%%time
chain.invoke({'pet': 'dog','breed':'shepherd'})

CPU times: total: 46.9 ms
Wall time: 5.16 s


AIMessage(content='Congratulations on your new shepherd! Shepherds are intelligent and eager to please, making them great candidates for training. Here are some tips to help you get started:\n\n1. **Start with Basic Commands**: Teach essential commands like "sit," "stay," "come," and "down." Use positive reinforcement, such as treats or praise, to encourage good behavior.\n\n2. **Be Consistent**: Use the same commands and gestures for each behavior. Consistency helps your dog understand what you expect', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bg6tfv1Fl0oDVlzD4QdKmxH5kFUeM', 'service_tier': '

In [31]:
%%time
chain.invoke({'pet': 'dragon','breed':'night fury'})

CPU times: total: 0 ns
Wall time: 3.33 s


AIMessage(content='Congratulations on adopting a Night Fury! While they are fictional creatures from the "How to Train Your Dragon" series, here are some fun and imaginative training tips inspired by the lore:\n\n1. **Build Trust**: Spend time with your Night Fury to build a bond. Offer treats (like fish or other dragon-friendly snacks) and speak softly to help them feel comfortable around you.\n\n2. **Create a Safe Space**: Ensure your dragon has a cozy and secure area to rest. Night Furies', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bg6tn6b4eWv3QYXDTwB2LDoJf07qD', 'service_tier': 'default', '

In [32]:
%%time
chain.batch([{'pet': 'dog','breed':'shepherd'}, {'pet': 'dragon','breed':'night fury'}])

CPU times: total: 0 ns
Wall time: 2.54 s


[AIMessage(content='Congratulations on your new shepherd! Shepherds are intelligent and eager to please, making them great candidates for training. Here are some tips to help you get started:\n\n### Basic Training Tips\n\n1. **Start with Basic Commands**: Teach essential commands like "sit," "stay," "come," "down," and "leave it." Use positive reinforcement, such as treats and praise, to encourage good behavior.\n\n2. **Consistency is Key**: Use the same commands and gestures consistently. This helps', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bg6tqun2SzHP1A8q5TzHWJht1Oh7D', 'service_tier': 'de

## Streaming

In [34]:
chat = ChatOpenAI(model='gpt-4o-mini', seed=365, temperature=0, max_tokens=10)

In [36]:
%%time
response = chain.stream({'pet': 'dragon','breed':'night fury'})

CPU times: total: 0 ns
Wall time: 0 ns


In [55]:
next(response)

AIMessageChunk(content=' Train', additional_kwargs={}, response_metadata={}, id='run--65405233-8ea8-4620-928c-4b8417712815')

In [56]:
for i in response:
    print(i.content)

 Your
 Dragon
"
 series
,
 here
 are
 some
 fun
 and
 imaginative
 training
 tips
 inspired
 by
 the
 lore
:


1
.
 **
Build
 Trust
**
:
 Spend
 time
 with
 your
 Night
 Fury
 to
 build
 a
 bond
.
 Offer
 treats
 (
like
 fish
 or
 meat
)
 and
 speak
 softly
 to
 help
 them
 feel
 comfortable
 around
 you
.


2
.
 **
Positive
 Rein
forcement
**
:
 Use
 rewards
 to
 encourage
 good
 behavior
.
 When
 your
 Night
 Fury
 follows
 a
 command
 or
 performs
 a
 trick
,



## The Runnable and Runnable Sequence Classes

In [62]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [57]:
chat_template = ChatPromptTemplate.from_messages([('human', "I've recently adopted a {pet}, which is a {breed}. Could you suggest some training tips?? \n")])

In [58]:
chat = ChatOpenAI(model='gpt-4o-mini', seed=365, temperature=0, max_tokens=100)

In [59]:
chain = chat_template | chat

In [65]:
RunnablePassthrough().invoke([1,2,3])

[1, 2, 3]

In [68]:
chat_template_tools = ChatPromptTemplate.from_template("""
What are the five most important tools a {job_title} needs?
Answer only by listing the tools.
""")

chat_template_strategy = ChatPromptTemplate.from_template("""
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
""")

In [70]:
chat_template_tools

ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job_title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [71]:
chat_template_strategy

ChatPromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, template='\nConsidering the tools provided, develop a strategy for effectively learning and mastering them:\n{tools}\n'), additional_kwargs={})])

In [72]:
string_parser = StrOutputParser()

In [82]:
chain_tools = chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()}
chain_strategy = chat_template_strategy | chat | string_parser

In [83]:
print(chain_tools.invoke({'job_title': 'data scientist'}))

{'tools': '1. Python\n2. R\n3. SQL\n4. Jupyter Notebook\n5. Git'}


In [84]:
print(chain_strategy.invoke({'tools': """
1. Python
2. R
3. SQL
4. Jupyter Notebook
5. TensorFlow/PyTorch
"""}))

To effectively learn and master Python, R, SQL, Jupyter Notebook, and TensorFlow/PyTorch, you can follow a structured strategy that includes setting clear goals, utilizing resources, practicing regularly, and applying your knowledge through projects. Here’s a step-by-step approach:

### Step 1: Set Clear Goals
- **Define Your Purpose**: Understand why you want to learn these tools. Are you interested in data analysis, machine learning, or data science? This will help you focus your


In [85]:
chain_combined = chain_tools | chain_strategy

In [87]:
print(chain_combined.invoke({'job_title': 'data scientist'}))

To effectively learn and master Python, R, SQL, Jupyter Notebook, and TensorFlow/PyTorch, you can follow a structured strategy that incorporates foundational knowledge, practical application, and continuous improvement. Here’s a step-by-step approach:

### Step 1: Establish a Learning Foundation

#### 1. Python
- **Resources**: Start with online courses (e.g., Codecademy, Coursera, or edX) that cover Python basics.
- **Practice**: Work on
